In [1]:
# Collect data

from pwn import *
from Crypto.Util.Padding import pad
from tqdm.auto import tqdm

def bxor(a, b):
    return bytes(i ^ j for i, j in zip(a, b))

r = process(["python", "aes_confusion.py"])
# r = remote("34.124.157.94", 19522)

# each encryption reveals 4 int32 of random state
# we need 624 int32s of state to recover the PRNG
# 180 batches gets 720 int32s
n_batches = 180
dummy_pln = b"\x00" * 15
dummy_pln_pad = pad(dummy_pln, 16)

# Send dummy plaintext for encryption in one batch
r.send(
    f"1\n{dummy_pln.hex()}\n".encode() * n_batches
)
dummy_encs = []
for _ in tqdm(range(n_batches)):
    r.recvuntil(b"Ciphertext: ")
    dummy_enc = bytes.fromhex(r.recvline().strip().decode())
    dummy_encs.append(dummy_enc)

# Decrypt ciphertexts and xor with plaintext to recover the IVs
r.send("".join(
    f"2\n{dummy_enc.hex()}\n"
    for dummy_enc in dummy_encs
).encode())
ivs = []
for _ in tqdm(range(n_batches)):
    r.recvuntil(b"Plaintext: ")
    iv_raw = bytes.fromhex(r.recvline().strip().decode())
    iv = bxor(dummy_pln_pad, iv_raw)
    ivs.append(iv)

# Obtain the flag ciphertext
r.send(b"3\n")
r.recvuntil(b"Flag: ")
flag = bytes.fromhex(r.recvline().strip().decode())
print(f"{flag.hex()=}")

[x] Starting local process '/home/sy/miniconda3/bin/python'
[+] Starting local process '/home/sy/miniconda3/bin/python': pid 34142


  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

flag.hex()='b0873e97866fe2f981db3d8b3f011517f2ee66e6f0aacb5daaa86a3f8b18718b'


In [2]:
from sage_army_knife import Untwister # template lib

ut = Untwister()

# clock PRNG 4 times when it generated key
for _ in range(4):
    ut.submit("?" * 32)
# clock PRNG 4 times when it generated iv
for _ in range(4):
    ut.submit("?" * 32)

# submit the data 
for iv in tqdm(ivs):
    for sli in range(0, 16, 4):
        ivsl = iv[sli:sli+4]
        ivsl_int = int.from_bytes(ivsl, "little")
        ivsl_bin = bin(ivsl_int)[2:].zfill(32)
        ut.submit(ivsl_bin)

# recover the PRNG at the start
rng = ut.get_random_after_seed()

  0%|          | 0/180 [00:00<?, ?it/s]

sat


In [3]:
from Crypto.Cipher import AES

key = rng.randbytes(16)
iv = rng.randbytes(16)
print(f"{key.hex()}")
print(f"{iv.hex()=}")

cipher = AES.new(key, mode=AES.MODE_CBC, iv=iv)
flag_pln = cipher.decrypt(flag)
print(f"{flag_pln=}")

476dceea6cb2c6aff5e8e58c2ada1451
iv.hex()='f7e5a5b42a34da9c7d0371e01d9bbf67'
flag_pln=b'grey{tr4v3ll1n9_84ck_1n_t1m3}\x03\x03\x03'
